## **AB Testing Meli notebook**

### importando librerias

Se importan las librerias a utilizar durate el proceso de exploración de datos, dentro de las librerias importamos ****[utilities.py](https://github.com/jmpaiba123/AB_Testing_Meli/blob/main/python_scripts/utilities.py)****, libreria construida específicamente para propósitos de nuestro proyecto.

In [1]:
# Preparamos el entorno
import pandas as pd
import numpy as np 
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import sys
import os

module_path = os.path.abspath(os.path.join('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/', 'python_scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utilities import AnalisisExperimento # type: ignore

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### **1. Lectura de la información compartida y control de registros**

In [2]:
# Cargamos los datos y creamos el dataframe a utilizar
df = pd.read_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/data/Inputs/Experiments DataSet For Excercise-small.csv', sep=",")
print(df.shape)
df.head()

(141553, 6)


,event_name,item_id,timestamp,site,experiments,user_id
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",3204901
1,PRODUCT,882352139.0,2021-08-02T23:55:51.673-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
2,PRODUCT,655266729.0,2021-08-02T23:56:16.083-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
3,PRODUCT,761520929.0,2021-08-02T23:56:29.989-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901
4,PRODUCT,757586409.0,2021-08-02T23:56:47.887-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901


In [3]:
# variable date, tipo fecha
df["datetime"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%f%z")
df["date"] = pd.to_datetime(df["datetime"]).dt.date

print("")
print("Registros por fecha:")
print(df.date.value_counts().sort_index())
print("")
unique_users_per_date = df.groupby('date')['user_id'].nunique()
print("Usuarios únicos por fecha: \n" + str(unique_users_per_date))
print("")
print(df.dtypes)


Registros por fecha:
date
2021-08-01     31094
2021-08-02    110459
Name: count, dtype: int64

Usuarios únicos por fecha: 
date
2021-08-01    2138
2021-08-02    6735
Name: user_id, dtype: int64

event_name                        object
item_id                          float64
timestamp                         object
site                              object
experiments                       object
user_id                            int64
datetime       datetime64[ns, UTC-04:00]
date                              object
dtype: object


In [4]:
print("Cantidad de registros: " + str(df.shape[0]))
print("Cantidad de usuarios únicos: " + str(df['user_id'].nunique()))
print("Cantidad de items únicos: " + str(df['item_id'].nunique()))
print("")
print("Secciones del sitio navegadas:")
print(df['event_name'].value_counts())
print(df['site'].value_counts())

Cantidad de registros: 141553
Cantidad de usuarios únicos: 7817
Cantidad de items únicos: 45273

Secciones del sitio navegadas:
event_name
SEARCH        76750
PRODUCT       58544
CHECKOUT_1     2115
CHECKOUT_2     1786
CHECKOUT_3     1270
BUY            1088
Name: count, dtype: int64
site
MLA    141553
Name: count, dtype: int64


Control de registros de las variables disponibles, tenemos un aproximado de 150k registros o navegaciones, para el transcurso de dos días 202108 (01-02), para 7,817 clientes. Eventualmente se observa que a medida se avanza en el proceso de compra las porciones de navegación disminuyes significativamente, donde 76750 fueron eventos relacionados con la búsqueda y solamente 1,088 de compra. 

In [5]:
df[df.event_name == "SEARCH"].head()

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",3204901,2021-08-02 23:55:38.966000-04:00,2021-08-02
6,SEARCH,NaN,2021-08-02T23:46:03.785-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:46:03.785000-04:00,2021-08-02
7,SEARCH,NaN,2021-08-02T23:46:39.972-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:46:39.972000-04:00,2021-08-02
8,SEARCH,NaN,2021-08-02T23:47:13.438-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:47:13.438000-04:00,2021-08-02
10,SEARCH,NaN,2021-08-02T23:39:56.973-0400,MLA,"{searchbackend/recommended-products=6157, sear...",402295,2021-08-02 23:39:56.973000-04:00,2021-08-02


In [6]:
df[df.event_name == "BUY"].head()

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
204,BUY,740440936.0,2021-08-02T23:53:46.363-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",426144,2021-08-02 23:53:46.363000-04:00,2021-08-02
295,BUY,680029039.0,2021-08-02T23:38:37.024-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:37.024000-04:00,2021-08-02
305,BUY,988145209.0,2021-08-02T23:41:31.971-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:31.971000-04:00,2021-08-02
447,BUY,885565729.0,2021-08-02T23:11:25.388-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",2866832,2021-08-02 23:11:25.388000-04:00,2021-08-02
562,BUY,934373429.0,2021-08-02T20:01:39.802-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",411939,2021-08-02 20:01:39.802000-04:00,2021-08-02


In [7]:
df[df.user_id == 3204901].sort_values(by='datetime').head(5)

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
63670,SEARCH,NaN,2021-08-02T12:53:11.057-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",3204901,2021-08-02 12:53:11.057000-04:00,2021-08-02
63671,SEARCH,NaN,2021-08-02T12:53:20.260-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 12:53:20.260000-04:00,2021-08-02
28364,SEARCH,NaN,2021-08-02T16:16:05.995-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 16:16:05.995000-04:00,2021-08-02
28365,PRODUCT,312247919.0,2021-08-02T16:16:32.788-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:16:32.788000-04:00,2021-08-02
28366,PRODUCT,343345729.0,2021-08-02T16:16:45.106-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:16:45.106000-04:00,2021-08-02


Imprimimos algunos de los registros de compra y búsqueda de nuestros clientes, y las primeras sesiones de navegación para un cliente cualquiera, observando que para los **event_name SEARCH o eventos de búsqueda no es posible identificar un item_id**, suceso de particular interés, puesto que para realizar seguimiento completo a la cadena de compra vamos a utilizar el item_id o artículo de interés.

Revisaremos en detalle algunos de los procesos de compra para nuestro clientes: 

In [8]:
df[df.user_id == 6498883].sort_values(by='datetime').loc[299:305]
#df[df.user_id == 426144].sort_values(by='datetime').loc[200:204]
#df[df.user_id == 466].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
299,SEARCH,NaN,2021-08-02T23:39:55.540-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02
300,PRODUCT,988145209.0,2021-08-02T23:39:56.097-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",6498883,2021-08-02 23:39:56.097000-04:00,2021-08-02
301,CHECKOUT_1,988145209.0,2021-08-02T23:40:30.328-0400,MLA,"{buyingflow/user-track=6796, buyingflow/escWeb...",6498883,2021-08-02 23:40:30.328000-04:00,2021-08-02
302,CHECKOUT_2,988145209.0,2021-08-02T23:40:40.135-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:40:40.135000-04:00,2021-08-02
303,CHECKOUT_2,988145209.0,2021-08-02T23:40:51.520-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:40:51.520000-04:00,2021-08-02
304,CHECKOUT_3,988145209.0,2021-08-02T23:41:18.970-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:18.970000-04:00,2021-08-02
305,BUY,988145209.0,2021-08-02T23:41:31.971-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:31.971000-04:00,2021-08-02


In [9]:
print(df[df.user_id == 6498883].experiments[299].strip('{}').split(', '))
print(df[df.user_id == 6498883].experiments[300].strip('{}').split(', '))
print(df[df.user_id == 6498883].experiments[301].strip('{}').split(', '))
print(df[df.user_id == 6498883].experiments[302].strip('{}').split(', '))
print(df[df.user_id == 6498883].experiments[303].strip('{}').split(', '))
print(df[df.user_id == 6498883].experiments[304].strip('{}').split(', '))

['searchbackend/recommended-products=6157', 'search/ungroup-products=3273', 'searchbackend/cbt-antiboost=6333', 'search/back-filters=5059', 'filters/sort-by-ranking=7057', 'search/best-seller-aa-testing-fail-fast-edition=4514', 'frontend/assetsCdnDomainMLU=DEFAULT', 'searchbackend/item-reputation=3824', 'search/remove-ecn-tag=4954', 'searchbackend/seller-reputation-change=4553', 'cookiesConsentBanner=DEFAULT', 'frontend/assetsCdnDomainMLA=DEFAULT', 'search/best-seller-fail-fast-edition-MLA=4916']
['search/remove-ecn-tag=4954', 'qadb/sa-on-vip=6696', 'vip/showV2V3BoxMessages=6430', 'search/tendency-landing-enabled-MLA=6980', 'pdp/viewItemPageMigrationDesktopReviewsNoTabs=4856', 'cookiesConsentBanner=DEFAULT', 'vip/shippingCalculatorMigrationModalExperiment=6551', 'vip/carousel-v2p-above-the-fold=6787', 'pdp/viewItemPageMigrationReturns=5208', 'vip/seller-with-tooltip=4692']
['buyingflow/user-track=6796', 'buyingflow/escWebMLA=2874']
['buyingflow/user-track=6796', 'buyingflow/address_hub

In [10]:
## eventos detectados durante la sesión de navegación Search
df[df.user_id == 6498883].experiments[299].strip('{}').split(', ')

['searchbackend/recommended-products=6157',
 'search/ungroup-products=3273',
 'searchbackend/cbt-antiboost=6333',
 'search/back-filters=5059',
 'filters/sort-by-ranking=7057',
 'search/best-seller-aa-testing-fail-fast-edition=4514',
 'frontend/assetsCdnDomainMLU=DEFAULT',
 'searchbackend/item-reputation=3824',
 'search/remove-ecn-tag=4954',
 'searchbackend/seller-reputation-change=4553',
 'cookiesConsentBanner=DEFAULT',
 'frontend/assetsCdnDomainMLA=DEFAULT',
 'search/best-seller-fail-fast-edition-MLA=4916']

In [11]:
## eventos detectados durante la sesión de navegación  Product
df[df.user_id == 6498883].experiments[300].strip('{}').split(', ')

['search/remove-ecn-tag=4954',
 'qadb/sa-on-vip=6696',
 'vip/showV2V3BoxMessages=6430',
 'search/tendency-landing-enabled-MLA=6980',
 'pdp/viewItemPageMigrationDesktopReviewsNoTabs=4856',
 'cookiesConsentBanner=DEFAULT',
 'vip/shippingCalculatorMigrationModalExperiment=6551',
 'vip/carousel-v2p-above-the-fold=6787',
 'pdp/viewItemPageMigrationReturns=5208',
 'vip/seller-with-tooltip=4692']

Validando el seguimiento de la cadena de compra de nuestro cliente **6498883** y su producto adquirido vía mercado libre **988145209**, confirmamos que :
 * eventos de **search** no tienen un ítem relacionado
 * dada la cantidad de eventos realizados durante la búsqueda, y el interés particular por determinar cual de las variantes aplicadas tuvo mayor éxito de compra, es necesario mapear o marcar cada uno de los mismos con una etiqueta positiva (1). 

### **2. Marcación de experimentos y variantes exitosas en cadena de compra**

Tomando en consideración todo lo mencionado anteriormente, y dando respuesta a la primera solicitud de nuestro ejercicio, buscaremos asignar de la manera más precisa, todos los eventos (y sus variantes correspondientes) involucrados en la cadena de cada una de las compras, tomando en consideración desde la navegación de búsqueda hasta la de compra.

In [12]:
# indentificacion de todas las compras ('BUY' event), considerando item_id & user_id
compras = df[df['event_name'] == 'BUY'][['item_id', 'user_id']].drop_duplicates()
dupla_compra = set(tuple(x) for x in compras.values)

# funcion para determinar si un registro se encuentra en la cadena de compra
def in_purchase_funnel(row):
    if pd.notna(row['item_id']):  # Only consider rows with a valid item_id
        return 1 if (row['item_id'], row['user_id']) in dupla_compra else 0
    return 0

# aplicamos la funcion a nuestro df, e insertamos el fla en una nueva columna
df['purchase_funnel_flag'] = df.apply(in_purchase_funnel, axis=1)

In [13]:
df[df.user_id == 6498883].sort_values(by='datetime').loc[299:305]

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date,purchase_funnel_flag
299,SEARCH,NaN,2021-08-02T23:39:55.540-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,0
300,PRODUCT,988145209.0,2021-08-02T23:39:56.097-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",6498883,2021-08-02 23:39:56.097000-04:00,2021-08-02,1
301,CHECKOUT_1,988145209.0,2021-08-02T23:40:30.328-0400,MLA,"{buyingflow/user-track=6796, buyingflow/escWeb...",6498883,2021-08-02 23:40:30.328000-04:00,2021-08-02,1
302,CHECKOUT_2,988145209.0,2021-08-02T23:40:40.135-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:40:40.135000-04:00,2021-08-02,1
303,CHECKOUT_2,988145209.0,2021-08-02T23:40:51.520-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:40:51.520000-04:00,2021-08-02,1
304,CHECKOUT_3,988145209.0,2021-08-02T23:41:18.970-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:18.970000-04:00,2021-08-02,1
305,BUY,988145209.0,2021-08-02T23:41:31.971-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:31.971000-04:00,2021-08-02,1


Acabamos de crear una variable **purchase_funnel_flag**, que nos permita asignar 1 a todas aquellas navegaciones **con información de item_id & user_id** relacionadas con la cadena de compra de algún artículo, o 0 en cualquier otro caso. Por otra parte, observando nuestro ejemplo anterior y la variable recientemente creada, la navegación de búsqueda, que posiblemente dio inicio a este proceso exitoso, no se marco positivamente, motivo por el cúal plantearemos un algoritmo adicional de identificación. Ordenaremos el dataframe por usuario y artículo de compra, encontraremos cada una de las búsquedas relacionadas con la cadena de compra (registro anterior: mismo usuario, mismo artículo, misma fecha; navegaciones del día anterior son no relacionadas) y asignaremos 1 a cada una de ellas.

 * Que hacer con cadenas de navegación interrumpidas por medía noche?

In [14]:
# organización del dataframe por usuario e item 
df_sorted = df.sort_values(by=['user_id', 'datetime']).reset_index(drop=True)
# vector booleano que contiene navegaciones actuales de éxito
mask = df_sorted['purchase_funnel_flag'] == 1

# creación de columnas, con data de las nvegaciones inmediatamente anteriores
df_sorted['prev_event'] = df_sorted['event_name'].shift(1)
df_sorted['prev_user'] = df_sorted['user_id'].shift(1).fillna(0).astype(int)
df_sorted['prev_date'] = df_sorted['date'].shift(1)

# marcamos ahora eventos anteriores que sean de búsqueda, de un mismo usuario, de una misma fecha y que su valor en la cadena de compra sea positivo
search_mask = (df_sorted['prev_event'] == 'SEARCH') & (df_sorted['user_id'] == df_sorted['prev_user']) & (df_sorted['date'] == df_sorted['prev_date']) & (df_sorted['purchase_funnel_flag'] == 1)
search_mask_1 = search_mask.shift(-1)

# Update purchase_funnel_flag where mask is True and search_mask is True
df_sorted.loc[mask | search_mask_1, 'purchase_funnel_flag'] = 1

In [15]:
#df_sorted[df_sorted.user_id == 466].sort_values(by='datetime')
df_sorted[df_sorted.user_id == 6498883].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date,purchase_funnel_flag,prev_event,prev_user,prev_date
117111,SEARCH,NaN,2021-08-01T20:47:19.528-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,PRODUCT,6498833,2021-08-02
117112,SEARCH,NaN,2021-08-01T20:54:02.302-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:54:02.302000-04:00,2021-08-01,0,SEARCH,6498883,2021-08-01
117113,CHECKOUT_1,680029039.0,2021-08-02T23:37:16.248-0400,MLA,"{buyingflow/user-track=6796, buyingflow/escWeb...",6498883,2021-08-02 23:37:16.248000-04:00,2021-08-02,1,SEARCH,6498883,2021-08-01
117114,CHECKOUT_2,680029039.0,2021-08-02T23:37:28.379-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:37:28.379000-04:00,2021-08-02,1,CHECKOUT_1,6498883,2021-08-02
117115,CHECKOUT_3,680029039.0,2021-08-02T23:38:14.079-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:14.079000-04:00,2021-08-02,1,CHECKOUT_2,6498883,2021-08-02
117116,BUY,680029039.0,2021-08-02T23:38:37.024-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:37.024000-04:00,2021-08-02,1,CHECKOUT_3,6498883,2021-08-02
117117,CHECKOUT_2,680029039.0,2021-08-02T23:39:08.456-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:08.456000-04:00,2021-08-02,1,BUY,6498883,2021-08-02
117118,CHECKOUT_2,680029039.0,2021-08-02T23:39:38.513-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:38.513000-04:00,2021-08-02,1,CHECKOUT_2,6498883,2021-08-02
117119,CHECKOUT_2,680029039.0,2021-08-02T23:39:42.170-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:42.170000-04:00,2021-08-02,1,CHECKOUT_2,6498883,2021-08-02
117120,SEARCH,NaN,2021-08-02T23:39:55.540-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,CHECKOUT_2,6498883,2021-08-02


In [16]:
# Eliminamos las columnas recientemente creadas
df_sorted.drop(columns=['prev_event','prev_user','prev_date'], inplace=True)

In [17]:
print(df.purchase_funnel_flag.value_counts())
print(df_sorted.purchase_funnel_flag.value_counts())

purchase_funnel_flag
0    134867
1      6686
Name: count, dtype: int64
purchase_funnel_flag
0    134068
1      7485
Name: count, dtype: int64


Finalmente hemos identificado positivamente las navegaciones involucradas en cada una de las cadenas de compra, inclusive las búsquedas. Del seguimiento de nuestro cliente tomado inicialmente, vemos que de sus dos compras hemos logrado identifcar toda su cadena de compra, contando las búsquedas o navegaciones señaladas a posteriori 799, son menores a la cantidad de compras iniciales 1.088, algo que nos da un parte de tranquilidad, puesto no hemos marcado búsquedas adicionales, pero **nos han faltado algúnas por etiquetar o los registros no se encontraban disponibles**. Y finalmente validamos que, después de los ajustes realizados nos mantenemos en 141.553 registros.

### **3. Apertura de las navegaciones, agrupaciones por experimento y variantes**

En la columna **experiments** se encuentra disponible toda la data de navegación, inicialmente crearemos diccionarios con la información allí contenida, algo que nos permite hacer mucho más fácil una apertura o explode de los experimentos, obteniendo así un registro para cada uno de los mismos. Nuevamente, nos será de gran utilidad nuestro paquete de funciones, construido en **[utilities.py](https://github.com/jmpaiba123/AB_Testing_Meli/blob/main/python_scripts/utilities.py)**

In [18]:
# Apply the function to the 'experiments' column and create a new column with the dictionaries
df_sorted['experiments_dict'] = df_sorted['experiments'].apply(AnalisisExperimento.str_to_dict)

# aplicamos funcion explode_dict para apertura de diccionarios (funcion creada y documentada en python_scripts/utilities)
df_exploded = df_sorted.apply(AnalisisExperimento.explode_dict, axis=1).reset_index(drop=True)
df_exploded = pd.concat(df_exploded.to_list(), ignore_index=True)
df_exploded.drop(columns=['experiments_dict','timestamp','experiments'], inplace=True)

# separamos columna experiment, para obtener su correspondiente descripción y variante
df_exploded[['path', 'experiment']] = df_exploded['experiment_key'].str.split('/', expand=True)
df_exploded.rename(columns={'experiment_value': 'variant'}, inplace=True)

In [19]:
# Control de registros para dataframe completo por experimentos
print("Info dataset:")
print(df_exploded.info())
print('\n')
print("Valores unicos por variable:")
print(df_exploded.nunique())

Info dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337265 entries, 0 to 1337264
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype                    
---  ------                --------------    -----                    
 0   event_name            1337265 non-null  object                   
 1   item_id               360385 non-null   float64                  
 2   site                  1337265 non-null  object                   
 3   user_id               1337265 non-null  int64                    
 4   datetime              1337265 non-null  datetime64[ns, UTC-04:00]
 5   date                  1337265 non-null  object                   
 6   purchase_funnel_flag  1337265 non-null  int64                    
 7   experiment_key        1337265 non-null  object                   
 8   variant               1337265 non-null  object                   
 9   path                  1337265 non-null  object                   
 10  experiment      

In [20]:
#df_exploded[(df_exploded.user_id == 466)&(df_exploded.experiment == 'search-list-algorithms')].sort_values(by='datetime')
#df_exploded[(df_exploded.user_id == 466)].sort_values(by='datetime')
df_exploded[(df_exploded.user_id == 6498883)].loc[1107118:1107149].sort_values(by='datetime')

,event_name,item_id,site,user_id,datetime,date,purchase_funnel_flag,experiment_key,variant,path,experiment
1107118,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,searchbackend/recommended-products,6157,searchbackend,recommended-products
1107130,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,search/best-seller-fail-fast-edition-MLA,4916,search,best-seller-fail-fast-edition-MLA
1107129,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,frontend/assetsCdnDomainMLA,DEFAULT,frontend,assetsCdnDomainMLA
1107128,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,cookiesConsentBanner,DEFAULT,cookiesConsentBanner,None
1107127,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,searchbackend/seller-reputation-change,4553,searchbackend,seller-reputation-change
1107125,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,searchbackend/item-reputation,3824,searchbackend,item-reputation
1107126,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,search/remove-ecn-tag,4954,search,remove-ecn-tag
1107123,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,search/best-seller-aa-testing-fail-fast-edition,4514,search,best-seller-aa-testing-fail-fast-edition
1107122,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,filters/sort-by-ranking,7057,filters,sort-by-ranking
1107121,SEARCH,NaN,MLA,6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1,search/back-filters,5059,search,back-filters


una vez listados todos los experimentos, tomaremos como punto de referencia [el contexto de meli](https://medium.com/mercadolibre-tech/a-b-testing-meli-3a5ad2b4594d) para agrupar y procesar los datos hasta acá logrados, dejando solamente la fecha (formato yyyy-mm-dd), nombre del experimento, variante correspondiente, user_id y el **valor máximo** de nuestra variable más relevante **purchase_funnel_flag**, lo que nos deja un data set que considera para cada cliente un registro único por experimento y su variante correspondiente, y un flag que nos indica 1 si fue exitoso en por lo menos alguna de sus variantes presentadas durante el día, y 0 en cualquier otro caso.

Lo anteriormente descrito se realiza con el fin de ajustarse de la manera más precisa posible al planteamiento de la prueba, sin embargo en este punto es posible tomar en cosideración lo siguiente:
 * **que pasa si para un usuario durante el mismo día, un experimento se ve involucrado en la compra de dos o más artículos diferentes?, utilizar var cant_item_id**
 * **que pasa si para un usuario durante el mismo día, un experimento se ve involucrado en la compra de dos o más artículos iguales?** item-usuario-hora

In [21]:
# Agrupamos con variables necesarias para conteo, asignacion de 0's a registros con item_id nulo
df_exploded['item_id'] = df_exploded['item_id'].fillna(0)
df_grouped = df_exploded.groupby(['date','experiment','variant','user_id','path']).agg({'purchase_funnel_flag': 'max',  'item_id': lambda x: x[df_exploded['purchase_funnel_flag'] != 0].nunique()}).reset_index()

In [22]:
#df_grouped[df_grouped.user_id == 466].sort_values(by='experiment')
df_grouped[df_grouped.user_id == 6498883].sort_values(by='date').head(20)

,date,experiment,variant,user_id,path,purchase_funnel_flag,item_id
1162,2021-08-01,ads-adsearch-boost-incremental-desktop-mla,3809,6498883,mclics,0,0
33297,2021-08-01,ungroup-products,3273,6498883,search,0,0
31845,2021-08-01,sort-by-ranking,7057,6498883,filters,0,0
28337,2021-08-01,show-pads-search-list,5146,6498883,mclics,0,0
22942,2021-08-01,seller-reputation-change,4553,6498883,searchbackend,0,0
21186,2021-08-01,search-pads-none-desktop-mla,3478,6498883,mclics,0,0
20018,2021-08-01,search-list-algorithms,5528,6498883,mclics,0,0
18052,2021-08-01,remove-ecn-tag,4954,6498883,search,0,0
27144,2021-08-01,show-pads-global,5176,6498883,mclics,0,0
14276,2021-08-01,item-reputation,3824,6498883,searchbackend,0,0


In [23]:
df_grouped_00 = df_grouped.groupby(['date', 'experiment', 'variant']).agg({'user_id': 'count', 'purchase_funnel_flag': 'sum'}).reset_index()
df_grouped_00 = df_grouped_00.rename(columns={"user_id": "participants" , "purchase_funnel_flag": "purchases"})
df_grouped_00['buy_rate'] = df_grouped_00 ['purchases']/df_grouped_00 ['participants']
df_grouped_00['buy_rate_percent'] = df_grouped_00['buy_rate'] .apply(lambda x: '{:.2f}%'.format(x*100))

In [24]:
print(df_grouped_00.shape)
df_grouped_00.head(25)

(143, 7)


,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
0,2021-08-01,HideTransitionModal,6646,2,0,0.000000,0.00%
1,2021-08-01,HideTransitionModal,6647,3,0,0.000000,0.00%
2,2021-08-01,HideTransitionModal,6993,5,0,0.000000,0.00%
3,2021-08-01,HideTransitionModal,DEFAULT,12,0,0.000000,0.00%
4,2021-08-01,address_hub,3574,189,143,0.756614,75.66%
5,2021-08-01,ads-adsearch-boost-incremental-desktop-mla,3809,1211,52,0.042940,4.29%
6,2021-08-01,assetsCdnDomainMLA,DEFAULT,1766,89,0.050396,5.04%
7,2021-08-01,assetsCdnDomainMLU,DEFAULT,1764,89,0.050454,5.05%
8,2021-08-01,back-filters,5059,1764,89,0.050454,5.05%
9,2021-08-01,back-filters,DEFAULT,1,0,0.000000,0.00%


In [25]:
df_grouped_00[df_grouped_00.experiment=='checkOnBehavior'].head(2)

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
19,2021-08-01,checkOnBehavior,348,22,0,0.000000,0.00%
20,2021-08-01,checkOnBehavior,349,21,1,0.047619,4.76%


In [26]:
print(df_grouped_00.groupby('date')['participants'].max())

date
2021-08-01    1845
2021-08-02    5962
Name: participants, dtype: int64


la cantidad máxima de participantes en cada uno de los días no excede nuestra cantidad de usuarios.

In [27]:
df_grouped_00.date = df_grouped_00.date.astype(str)

Hemos llegado a la parte final del procesamiento de datos, y tenemos dos dataframes importantes, uno donde podemos encontrar la información agrupada por experimento, fecha, variante y resultados para cada uno de ellos, otro dataframe bastante útil, donde tenemos un apertura adicional por cliente o usuario, que nos permitira tomar información más detallada y desarrollar las pruebas estadísticas pertinentes, para cada uno de los casos.

In [28]:
df_grouped.to_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/data/Outputs/data_test.csv', sep = ",", index = False)
df_grouped_00.to_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/python_scripts/grouped_inf.csv', sep = ",", index = False)
df_grouped_00.to_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/data/Outputs/grouped_inf.csv', sep = ",", index = False)

In [29]:
# Cantidad única de experimentos 
unique_experiments = df_grouped_00["experiment"].nunique()
print(f"Cantidad única de experimentos : {unique_experiments}\n")
# cantidad única de experimentos por día
experiments_per_day = df_grouped_00.groupby("date")["experiment"].nunique().reset_index()
print("Cantidad única de experimentos por día:")
print(experiments_per_day.to_string(index=False))

Cantidad única de experimentos : 44

Cantidad única de experimentos por día:
      date  experiment
2021-08-01          43
2021-08-02          44


https://learn.microsoft.com/en-us/azure/developer/python/tutorial-containerize-simple-web-app-for-app-service?tabs=web-app-flask

http://pruebamelijmpaiba.azurewebsites.net/experiment/sa-on-vip/result?day=2021-08-01